### DB 연결

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


### 테이블확인

In [3]:
curs.execute("select * from mimiciv_hosp.d_icd_diagnoses;")
columns_name = [desc[0] for desc in curs.description]
d_icd_diagnoses = pd.DataFrame(curs.fetchall(), columns=columns_name) 
d_icd_diagnoses

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [11]:
filtered_diagnosislist = d_icd_diagnoses >> mask(X.long_title.str.contains('hronic obstructive pulmonary disease'))
filtered_diagnosislist

,icd_code,icd_version,long_title
26416,J44,10,Other chronic obstructive pulmonary disease
26417,J440,10,Chronic obstructive pulmonary disease with (ac...
26418,J441,10,Chronic obstructive pulmonary disease with (ac...
26419,J449,10,"Chronic obstructive pulmonary disease, unspeci..."


In [ ]:
#icd_code J44, J440, J441, J449

In [13]:
curs.execute("select * from mimiciv_hosp.diagnoses_icd;")
columns_name = [desc[0] for desc in curs.description]
diagnoses_icd = pd.DataFrame(curs.fetchall(), columns=columns_name) 
diagnoses_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [14]:
diagnoses_icd >> mask(X.icd_code.str.contains('J44'))

,subject_id,hadm_id,seq_num,icd_code,icd_version
653,10001884,21192799,1,J441,10
676,10001884,22532141,1,J441,10
693,10001884,24746267,1,J441,10
710,10001884,24962904,1,J441,10
750,10001884,26184834,1,J441,10
...,...,...,...,...,...
4753588,19994379,23099193,23,J449,10
4754459,19996406,29048379,2,J449,10
4755929,19999204,29046609,10,J449,10
4756264,19999828,25744818,10,J449,10


In [15]:
curs.execute("select * from mimiciv_hosp.diagnoses_icd where icd_code in ('J44', 'J440', 'J441', 'J449');")
columns_name = [desc[0] for desc in curs.description]
copd_diag = pd.DataFrame(curs.fetchall(), columns=columns_name) 
copd_diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10001884,21192799,1,J441,10
1,10001884,22532141,1,J441,10
2,10001884,24746267,1,J441,10
3,10001884,24962904,1,J441,10
4,10001884,26184834,1,J441,10
...,...,...,...,...,...
14065,19987702,28881038,6,J449,10
14066,19990366,24092667,6,J449,10
14067,19992875,21570862,17,J449,10
14068,19992875,23210485,16,J449,10


In [17]:
copd_diag.to_csv('../outputs/copd_diag.csv', index=False, encoding='utf-8-sig')
copd_diag = pd.read_csv('../outputs/copd_diag.csv')

In [18]:
copd_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14070 entries, 0 to 14069
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subject_id   14070 non-null  int64 
 1   hadm_id      14070 non-null  int64 
 2   seq_num      14070 non-null  int64 
 3   icd_code     14070 non-null  object
 4   icd_version  14070 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 549.7+ KB


In [19]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              9003 non-null   int64         
 1   subject_id              9003 non-null   int64         
 2   hadm_id                 9003 non-null   int64         
 3   stay_id                 9003 non-null   int64         
 4   int_stayid              7860 non-null   float64       
 5   admittime               9003 non-null   datetime64[ns]
 6   intubationtime          7860 non-null   datetime64[ns]
 7   int_itemid              7860 non-null   float64       
 8   int_weight              7860 non-null   float64       
 9   ext_stayid              6825 non-null   float64       
 10  extubationtime          8759 non-null   datetime64[ns]
 11  ext_itemid              6825 non-null   float64       
 12  ext_weight              6825 non-null   float64 

In [20]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [21]:
sub1_copd = sub1list >> left_join(copd_diag, by=['subject_id','hadm_id'])

In [22]:
sub1_copd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9039 entries, 0 to 9038
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   subject_id   9039 non-null   int64  
 1   hadm_id      9039 non-null   int64  
 2   stay_id      9039 non-null   int64  
 3   seq_num      562 non-null    float64
 4   icd_code     562 non-null    object 
 5   icd_version  562 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 423.8+ KB


In [23]:
#sub1_copd.to_csv('../outputs/sub1_copd.csv', index=False, encoding='utf-8-sig')